# Assignment 4

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import data

In [2]:
# Import data
D1 = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=1, usecols=[0,35])

D2 = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/UVicSci_temperature.dat', 
                 header=2)

In [3]:
# Convert time in D1 from MatLab time to Python Time
D1['Time'] = D1['NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
D12 = D1.rename(index=str, 
                columns={"NaN": "MatLab Time", "48.4623": "Temperature"})

In [4]:
# Reorder columns 
cols = D12.columns.tolist()
cols = cols[-1:] + cols[:-1]
D123 = D12[cols]

# Set time as index column
DH = D123.set_index('Time')

In [5]:
# Generate dates for D2 (minute resolution data)
date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     end='2017-08-30 16:59:00.000000', 
                     freq='min')

# Insert dates into D1 dataframe
D2.insert(loc=0, column='Time', value=date)

# Rename the columns
D22 = D2.rename(index=str, columns={"2979360": "Temperature"})

# Set index
DM = D22.set_index('Time')

In [6]:
# Select the dates:
# Hour resolution data
DH1 = DH.loc['2015-12-01 00:00':'2016-03-01 23:00']['Temperature']
DH2 = DH.loc['2016-06-01 00:00':'2016-09-01 23:00']['Temperature']

# Minuite resolution data
DM1 = DM.loc['2015-12-01 00:00':'2016-03-01 23:59']['Temperature']
DM2 = DM.loc['2016-06-01 00:00':'2016-09-01 23:59']['Temperature']

### Relevant functions 

In [7]:
# Inverse chi-squared distribution
df = 2
CI = 0.95
bounds = [(1-CI)/2, 1-(1-CI)/2]

chiT = df / (2 * sci.stats.chi2.ppf(bounds[0], df))
chiB = df / (2 * sci.stats.chi2.ppf(bounds[1], df))

In [8]:
# Design the filter
def ButterBP(CutL, CutH, fs, order):
    fn = fs/2
    low = CutL / fn
    high = CutH / fn
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def ButterBPF(x, CutL, CutH, fs, order):
    b, a = ButterBP(CutL, CutH, fs, order)
    y = filtfilt(b, a, x)
    return y

def GraphButter(CutL, CutH, fs, order, worN=2000):
    b, a = ButterBP(CutL, CutH, fs, order)
    w, h = freqz(b, a, worN)
    return b, a, w, h

## Question 1: Power spectral density

### a) Hour resolution: Power spectral density $S_{yy}$ based on periodgrams using Hanning window with $50\%$ overlap

In [9]:
steps_a = len(DH1)
# dTa = 3600
# FSa = np.fft.rfftfreq(steps_a, dTa)

In [10]:
# Generate the power spectral density 
N1a = 2

F1a, G1a = welch(DH1, 1/3600, nperseg = steps_a/N1a,\
            window = sci.signal.windows.hann(int(steps_a/N1a)),\
            noverlap = steps_a/(2*N1a), nfft = steps_a/N1a, detrend=False,\
            return_onesided=True, scaling = 'density')

F2a, G2a = welch(DH2, 1/3600, nperseg = steps_a/N1a,\
            window = sci.signal.windows.hann(int(steps_a/N1a)),\
            noverlap = steps_a/(2*N1a), nfft = steps_a/N1a, detrend=False,\
            return_onesided=True, scaling = 'density')

In [11]:
# Generate the confidence limits 
G1Ta = G1a * chiT * 4/3
G1Ba = G1a * chiB * 4/3

G2Ta = G2a * chiT * 4/3
G2Ba = G2a * chiB * 4/3

In [12]:
# Plot it out 
fig, (ax1a, ax2a) = plt.subplots(2, 1, figsize=(10, 8))

title1a = r'Power spectral density of UVicSCI temperature data from 1 Dec. 2015 to 1 Mar. 2016 using the Hanning window with ${0}\%$ confidence intervals, and NFFT of 2^{1}. Hour resolution.'.format(CI*100, int(np.log2(steps_a/N1a)))
ax1a.loglog(F1a, G1Ta, 'b--', F1a, G1Ba, 'b--', alpha = 0.5)
ax1a.loglog(F1a, G1a)
ax1a.set_title("\n".join(wrap(title1a, 90)))
ax1a.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax1a.set_xlabel(r'Frequency [$Hz$]')

title2a = r'Power spectral density of UVicSCI temperature data from 1 June 2016 to 1 Sept. 2016 using the Hanning window with ${0}\%$ confidence intervals, and NFFT of 2^{1}. Hour resolution.'.format(CI*100, int(np.log2(steps_a/N1a)))
ax2a.loglog(F2a, G2Ta, 'b--', F2a, G2Ba, 'b--', alpha = 0.5)
ax2a.loglog(F2a, G2a)
ax2a.set_title("\n".join(wrap(title2a, 90)))
ax2a.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax2a.set_xlabel(r'Frequency [$Hz$]')

fig.tight_layout()
plt.show()

<Figure size 720x576 with 2 Axes>

### b) Plot power spectral density of summer and winter data on same figure in variance preserving form 

### c) Minuite resolution: Power spectral density $S_{yy}$ based on periodgrams using Hanning window with $50\%$ overlap

In [13]:
steps_c = int(len(DM1)/4)
# dTc = 60
# FSc = np.fft.rfftfreq(steps_c, dTc)

In [14]:
# Generate the power spectral density 
N1c = 4

# F1c, G1c = welch(DM1, 1/60, nperseg = steps_c/N1c,\
#             window = sci.signal.windows.hann(int(steps_c/N1c)),\
#             noverlap = steps_c/(2*N1c), nfft = steps_c/N1c, detrend=False,\
#             return_onesided=True, scaling = 'density')

F1c, G1c = periodogram(DM1, 1/60, \
            window=sci.signal.windows.hann(int(steps_c/N1c)),\
            nfft = int(steps_c/N1c), detrend=False,\
            return_onesided=True, scaling = 'density')

F2c, G2c = periodogram(DM2, 1/60, \
            window=sci.signal.windows.hann(int(steps_c/N1c)),\
            nfft = int(steps_c/N1c), detrend=False,\
            return_onesided=True, scaling = 'density')

/Users/Kev/anaconda3/lib/python3.7/site-packages/scipy/signal/spectral.py:278: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[s]


In [15]:
# Generate the confidence limits 
G1Tc = G1c * chiT * 4/3
G1Bc = G1c * chiB * 4/3

G2Tc = G2c * chiT * 4/3
G2Bc = G2c * chiB * 4/3

In [16]:
# Plot it out 
fig, (ax1c, ax2c) = plt.subplots(2, 1, figsize=(10, 8))

title1c = r'Power spectral density of UVicSCI temperature data from 1 Dec. 2015 to 1 Mar. 2016 using the Hanning window with ${0}\%$ confidence intervals and NFFT of 2^{1}. Minuite resolution.'.format(CI*100, int(np.log2(steps_c/N1c)))
ax1c.loglog(F1c, G1Tc, 'b--', F1c, G1Bc, 'b--', alpha = 0.5)
ax1c.loglog(F1c, G1c)
ax1c.set_title("\n".join(wrap(title1c, 100)))
ax1c.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax1c.set_xlabel(r'Frequency [$Hz$]')

title2c = r'Power spectral density of UVicSCI temperature data from 1 June 2016 to 1 Sept. 2016 using the Hanning window with ${0}\%$ confidence intervals and NFFT of 2^{1}. Minuite resolution.'.format(CI*100, int(np.log2(steps_c/N1c)))
ax2c.loglog(F2c, G2Tc, 'b--', F2c, G2Bc, 'b--', alpha = 0.5)
ax2c.loglog(F2c, G2c)
ax2c.set_title("\n".join(wrap(title2c, 100)))
ax2c.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax2c.set_xlabel(r'Frequency [$Hz$]')

fig.tight_layout()
plt.show()

<Figure size 720x576 with 2 Axes>

## Question 2: Filter design

### a) Deriving a frequency-domain Butterworth high-pass filter with attenuation at $0.9\omega_c$ that is $-30 \ dB$ compared to the attenuation at $\omega_c$ 

Decibels: $dB = 10 \log \left(\frac{I}{I_0}\right)$ where $I$ is the intensity and $I_0$ is the refrerence intensity

Butterworth filter: $H(\omega) = [1 + \left(\frac{\omega_0}{\omega}\right)^{2n}]^{-\frac{1}{2}}$

Need to find $n$ (the order of the Butterworth filter):

$$\therefore -30 = 10 \log \left(\frac{|H(\omega_c)|^2}{|H(0.9 \omega_c)|^2}\right) = 10 \log \left(\frac{1 + \left(\frac{\omega_c}{\omega_c}\right)^{2n}}{1 + \left(\frac{0.9\omega_c}{\omega_c}\right)^{2n}}\right)$$
$$\Rightarrow -3 = \log \left(\frac{2}{1 + \left(\frac{10}{9}\right)^{2n}}\right)$$
$$\begin{align}
10^{-3} &= \frac{2}{1 + \left(\frac{10}{9}\right)^{2n}} \\
\left(\frac{10}{9}\right)^{2n} &= 2\times 10^3 - 1 \\
2n \ln{\left(\frac{10}{9}\right)} &= \ln(2\times 10^3 - 1)
\end{align}$$

$$\therefore n = \frac{\ln\left(2\times 10^3 - 1\right)}{2 \ln{\left(\frac{10}{9}\right)}} \approx 36$$

### b) Sketch $H(\omega)$

In [17]:
omega = np.linspace(-2, 2, 400)

H = (1 + omega**(-72)) ** -0.5
rec_high = np.zeros(len(omega))
rec_high[0: int(len(omega)/4)] = 1
rec_high[int(3*len(omega)/4): ] = 1

In [18]:
plt.figure(figsize=(10, 4))
plt.plot(omega, rec_high, '--', label=r'Rectangular filter')
plt.plot(omega, H, label=r'$H(\omega)$')
plt.plot([-2, 2], [sqrt(0.5), sqrt(0.5)], '--', label=r'$\sqrt{0.5}$')
plt.xlabel(r'$\omega$')
plt.ylabel(r'$H(\omega)$')
plt.title(r'Butterworth high-pass filter with $-30dB$ attenuation at $0.9\omega_c$ compared to attenuation at $\omega_c$')
plt.grid(True)
plt.legend()

<Figure size 720x288 with 1 Axes>

### c) Contrast, compare, and explain Butterworth vs rectangular filter



## Question 3: PSD and filtering of synthetic data

$$x(t) = \cos(22 \pi t) + 0.7\sin(14 \pi t) + 0.5\sin(147 \pi t)$$


In [19]:
steps3 = 3*10**4
range3 = 15
t3 = np.linspace(-range3, range3, steps3, endpoint=False)

In [20]:
# Define x(t)
x1 = cos(22 * pi * t3)
x2 = 0.7 * sin(14 * pi * t3)
x3 = 0.5 * sin(147 * pi * t3)

x = x1 + x2 + x3

### a) Plot $x(t)$

In [21]:
# Plot it out 
plt.figure(figsize=(10, 4))
plt.plot(t3, x)
plt.title(r'$x(t)$ from $t=-1$ to $t=1$')
plt.xlabel(r'Time ($s$)')
plt.ylabel(r'$x(t)$')
plt.xlim(-1, 1)

plt.show()

<Figure size 720x288 with 1 Axes>

### b) Plot power density spectrum of $x(t)$

In [22]:
# Get the power spectral density 
Fx, Gx = welch(x, steps3/2, nperseg=steps3,\
            window=sci.signal.windows.hann(int(steps3)),\
            noverlap = steps3/2, nfft = steps3, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [23]:
# Plot it out 
plt.figure(figsize=(10, 4))
plt.loglog(Fx, Gx)
plt.title(r'Power density spectrum of $x(t)$')
plt.xlabel(r'Frequency ($Hz$)')
plt.ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')

plt.show()

<Figure size 720x288 with 1 Axes>

### c) Recover $x_1 (t)$ from $x(t)$ using a Butterworth filter

In [24]:
# Butter up x(t)
fig, (f3d1, f3d2) = plt.subplots(2, 1, figsize=(10, 8))

CutL = 10
CutH = 16
order = 4
fs3 = steps3/(2*range3)
f1 = ButterBPF(x, CutL, CutH, fs3, order)

# Plot the filter
b, a, w, h = GraphButter(CutL, CutH, fs3, order)
f3d1.plot(fs3/(2*pi) * w, abs(h), label="Order = {0}".format(order))
f3d1.plot([0, fs3/2], [sqrt(0.5), sqrt(0.5)], '--', label=r'$\sqrt{0.5}$')
f3d1.set_title('Butterworth filter of order {0}'.format(order))
f3d1.set_xlabel(r'Frequency ($Hz$)')
f3d1.set_ylabel(r'$H(\omega)$')
f3d1.grid(True)
f3d1.legend(loc='best')
f3d1.set_xlim(0, 30)

# Plot filtered x(t)
f3d2.plot(t3, x1, label=r'$x_1$')
f3d2.plot(t3, f1, label=r'$|H(\omega)|$ (Frequency responce of filter)')
title3b = r'Isolation of $x1=\cos(22\pi t)$ signal from x(t) via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
f3d2.set_title("\n".join(wrap(title3b, 90)))
f3d2.set_xlabel(r'Time ($s$)')
f3d2.set_ylabel(r'$|H(\omega)|$')
f3d2.legend(loc=1)
f3d2.set_xlim(-1, 1)

fig.tight_layout()
plt.show()

/Users/Kev/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


<Figure size 720x576 with 2 Axes>

### d) Power density spectra of filtered time series $x_f (t)$

In [25]:
# Power spectra of the filtered signal 
Ff1, Gf1 = welch(f1, steps3/2, nperseg=steps3,\
            window=sci.signal.windows.hann(int(steps3)),\
            noverlap = steps3/2, nfft = steps3, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [27]:
# Plot it out 
plt.figure(figsize=(10, 4))
plt.loglog(Fx/(2*pi), Gx, label = r'PDS of $x(t)$')
plt.loglog(Ff1/(2*pi), Gf1, label = r'PDS of $x_f(t)$')
plt.title(r'Power density spectrum of $x(t)$ and $x_f(t)$ (filtered signal)')
plt.xlabel(r'Frequency ($Hz$)')
plt.ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
plt.legend()

plt.show()

<Figure size 720x288 with 1 Axes>

### e) Difference between filtered time series and $x_1 (t)$: $x_f (t) - x_1 (t)$

In [28]:
# Plot differnce between recoverd and actual signal
fig, (r1, r2) = plt.subplots(2, 1, figsize=(10, 8))
r1.plot(t3, f1-x1)
title3e1 = r'Difference between $x1=\cos(22\pi t)$ and the isolated signal $x_f(t)$ via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
r1.set_title("\n".join(wrap(title3e1, 90)))
r1.set_xlabel(r'Time ($s$)')
r1.set_ylabel(r'$x_f(t) - x_1(t)$')

r2.plot(t3, f1-x1)
title3e2 = r'Magnified plot of the difference between $x1=\cos(22\pi t)$ and the isolated signal $x_f(t)$ via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
r2.set_title("\n".join(wrap(title3e2, 90)))
r2.set_xlabel(r'Time ($s$)')
r2.set_ylabel(r'$x_f(t) - x_1(t)$')
r2.set_xlim(-1, 1)
r2.set_ylim(-0.016, 0.016)

fig.tight_layout()
plt.show()

<Figure size 720x576 with 2 Axes>

### f) Create a rectangular filter to isolate $x_1 (t)$ from $x(t)$

In [29]:
# Design a square filter 
def square(F, CutL, CutH):
    sq = np.zeros(len(F))
    sp1 = np.min(np.where(CutL < F/(4*pi)))
    sp2 = np.max(np.where(F/(4*pi) < CutH))
    sq[sp1:sp2] = 1
    flip = np.flip(sq)
    sqM = np.append(flip[:-1], sq[1:])
    return sq, sqM

# Plot the square filter 
plt.figure(figsize=(10, 4))
plt.semilogx(Ff1, square(Ff1, CutL, CutH)[0])
plt.xlabel(r'Frequency ($Hz$)')
plt.ylabel(r'$H_R(\omega)$')
plt.title(r'Rectangular filter $H_R(\omega)$ with cut-off frequencies at {0}$Hz$ and {1}$Hz$'.format(CutL, CutH))

plt.show()

<Figure size 720x288 with 1 Axes>

In [33]:
# Filtered signal using the rectangular filter 
f2 = Gx*square(Ff1, CutL, CutH)[0]

# Plot it out 
plt.figure(figsize=(10, 4))
plt.loglog(Ff1/(2*pi), Gx, label='PSD of x(t)')
plt.loglog(Ff1/(2*pi), f2, label='Filtered region')
plt.xlabel(r'Frequency ($Hz$)')
plt.ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
plt.title(r'Power spectral density of $x(t)$ and region filtered by the rectangular filter')

plt.legend()
plt.show()

<Figure size 720x288 with 1 Axes>

In [34]:
f3Freq = np.append(f2[1:0], np.flip(f2))

plt.figure(figsize=(10, 4))
f3 = np.fft.ifft(f3Freq)
plt.plot(f3)
# plt.xlim(-1, 1)
plt.show()

# plt.loglog(f2)

# # f4 = np.fft.ifft(np.fft.fft(x)*square(Fy1, CutL, CutH)[1])
# # plt.semilogx(np.fft.fft(x))
# plt.semilogx(f4)

/Users/Kev/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py:501: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


<Figure size 720x288 with 1 Axes>

In [36]:
# plt.plot(square(Fy1, CutL+2, CutH-1))
# flip = np.flip(square(Fy1, CutL+2, CutH-1))
flippy = square(Ff1, CutL+2, CutH-1)[1]
# plt.semilogy(np.fft.fft(x)*flippy)
# plt.plot(x)
plt.figure(figsize=(10, 4))
# plt.plot(x1)
plt.plot(np.fft.ifft(np.fft.fft(x)*flippy))
# plt.plot(np.fft.ifft(Gx*square(Fy1, CutL+2, CutH-1)[0]))
plt.xlim(0, 1000)

plt.figure(figsize=(10, 4))
plt.plot(np.fft.ifft(np.fft.fft(x)*flippy)-x1)
# plt.xlim(0, 1000)

<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>